In [61]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

import datetime

# postgres pull

db_password = "postgres"
db_user = "postgres"
db_name = "Youtube P3"
endpoint = 'youtube.cb1bticre0py.us-east-1.rds.amazonaws.com'

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

final_model = pd.read_sql('SELECT * FROM final_model;', con = engine)


In [62]:
final_model.head()


,index,category_e,publish_day_num,publish_to_trend,pt_views,pt_likes,pt_dislikes,pt_comments,target
0,0,3.0,2,3.0,5173063.0,1052986.0,29927.0,63557.0,0
1,1,8.0,0,2.0,267945.0,3040.0,394.0,1226.0,0
2,2,12.0,1,1.0,81049.0,5684.0,207.0,422.0,0
3,3,5.0,0,2.0,872005.0,52371.0,491.0,5043.0,0
4,4,7.0,1,6.0,573902.0,69319.0,970.0,3311.0,1


In [63]:
#slimmer_unique=pd.read_csv('https://elasticbeanstalk-us-west-1-140033263198.s3.us-west-1.amazonaws.com/cleaned.csv')  

In [64]:
from sklearn.model_selection import train_test_split
import sklearn as skl

In [65]:
y = final_model['target'].to_numpy()
X = final_model[['category_e','publish_to_trend','publish_day_num','pt_views','pt_likes',
                        'pt_dislikes','pt_comments']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [79]:
X_test

X_scaler





StandardScaler()

In [66]:
import tensorflow as tf
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.wrappers.scikit_learn import KerasClassifier

In [67]:
#Create a Keras Sequential model and add more than one Dense hidden layer
nn_model = tf.keras.models.Sequential()

nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=7))

nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))

nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                80        
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [68]:
# Compile the model and train over more than 100 epochs
loss_fn = keras.losses.mean_squared_error
nn_model.compile(loss=loss_fn, optimizer="adam", metrics=["accuracy"])
#nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
5652/5652 [==============================] - 9s 1ms/step - loss: 0.0514 - accuracy: 0.9369
Epoch 2/5
5652/5652 [==============================] - 8s 1ms/step - loss: 0.0456 - accuracy: 0.9424
Epoch 3/5
5652/5652 [==============================] - 8s 1ms/step - loss: 0.0454 - accuracy: 0.9424
Epoch 4/5
5652/5652 [==============================] - 8s 1ms/step - loss: 0.0453 - accuracy: 0.9424
Epoch 5/5
5652/5652 [==============================] - 9s 2ms/step - loss: 0.0451 - accuracy: 0.9425


In [69]:
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                80        
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


In [81]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset.
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1884/1884 - 3s - loss: 0.0445 - accuracy: 0.9435 - 3s/epoch - 2ms/step
Loss: 0.04451049864292145, Accuracy: 0.9435117244720459


In [71]:
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# print(confusion_matrix(y_test,y_pred))
X_test_scaled

array([[-8.55082750e-01,  1.30912364e-01, -4.88367740e-01, ...,
        -2.92002204e-01,  7.92172148e-02, -6.04077476e-02],
       [ 9.90856882e-01,  1.30912364e-01,  1.52667988e+00, ...,
         6.57966178e-03, -6.15537674e-02,  2.69573393e-02],
       [-6.39657650e-02, -1.97857550e-01,  1.52667988e+00, ...,
        -3.16950628e-01, -7.37817552e-02, -1.42011885e-01],
       ...,
       [ 1.78197387e+00,  1.30912364e-01, -1.49589155e+00, ...,
        -2.70386897e-01, -2.84487271e-02, -1.45588999e-03],
       [ 2.04567953e+00,  1.30912364e-01,  1.02291797e+00, ...,
        -2.15985347e-01, -5.64836261e-02, -1.25568008e-01],
       [ 9.90856882e-01,  2.76107168e+00, -4.88367740e-01, ...,
        -3.00775476e-01, -4.59456853e-02, -1.20590099e-01]])

In [72]:
# # save the model
from pickle import dump, load
import joblib
from sklearn.metrics import accuracy_score
# dump(nn_model, open('./models/model.pkl', 'wb'))

# Use H5 format since Pkl format was giving issues
nn_model.save('model_nn.h5')
#Used Pkl for model columns
model_columns = list(X_test.columns)
dump(model_columns, open('model_columns_nn.pkl', 'wb'))

# save the scaler using pickle format 
dump(X_scaler, open('scaler_nn.pkl', 'wb'))



In [73]:
model_columns



['category_e',
 'publish_to_trend',
 'publish_day_num',
 'pt_views',
 'pt_likes',
 'pt_dislikes',
 'pt_comments']

In [74]:
#load the h5 model 
reconstructed_model=keras.models.load_model('model_nn.h5')

# load the scaler uing pickle format.
scaler = joblib.load('./scaler_nn.pkl')
#load the columns usinf pickle format
columns = joblib.load ('./model_columns_nn.pkl')



In [75]:
#Uncomment if we are only inputting single row of data to the model
# index=1000

In [76]:
#Uncomment the bottom 3 lines and comment the lines 4 and 5 if we habe single input row to the model

# test_input=X_test.iloc[index,:].values.tolist()
# test_input=[[0,6, 6, 47, 54, 3, 5, 7, 0.5]]
# test_target=y_test[index]
test_input=X_test
test_target=y_test
len(test_input)



60278

In [77]:
#Uncomment the line below and comment the 2nd  line below if we have just one row as input
# test_input_df = pd.DataFrame ([test_input], columns = columns)
test_input_df=test_input


test_input_scaled = scaler.transform(test_input_df)

# reconstructed_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# score = reconstructed_model.evaluate(test_input, test_target, verbose=0)
# print("%s: %.2f%%" % (reconstructed_model.metrics_names[1], score[1]*100))

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
# reconstructed_model.fit(test_input_scaled, test_target)
# score = reconstructed_model.evaluate(test_input_scaled, test_target, verbose=0)
# print("%s: %.2f%%" % (reconstructed_model.metrics_names[1], score[1]*100))

test_target_hat = reconstructed_model.predict(test_input_scaled)
# evaluate accuracy



print(test_target_hat)

len(test_target_hat)
test_target_hat[test_target_hat > 0.5] = 1
test_target_hat[test_target_hat <= 0.5] = 0


print(test_target_hat)
print(test_target)

acc = accuracy_score(test_target, test_target_hat)
print('Test Accuracy:', acc)

1884/1884 [==============================] - 3s 1ms/step
[[0.8053878 ]
 [0.8736482 ]
 [0.09864715]
 ...
 [0.7774712 ]
 [0.7755364 ]
 [0.9998369 ]]
[[1.]
 [1.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]
[1 1 0 ... 1 1 1]
Test Accuracy: 0.9435117289890176
